In [110]:
import pandas as pd
from datetime import datetime, timedelta

import json as json
from pandas.io.json import json_normalize

import numpy as np
import os
import scipy.optimize as sco

import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

In [111]:
array_dataframe = []
dados = pd.DataFrame([])
scoins = []

folder_candle = 'Operacoes'

for folder in os.scandir(folder_candle):
    scoins.append(folder.name)
    for entry in os.scandir(folder):
        if entry.is_file():
            coin_symbol = (os.path.splitext(entry.name)[0])        
            with open(entry.path) as d:            
                coin_hist = json.load(d);
            df = json_normalize(coin_hist);
            
            if not df.empty:
            
                df['coin'] = folder.name
                df['amount'] = pd.to_numeric(df['amount'].replace('.',','))
                df['rate'] = pd.to_numeric(df['total'].replace('.',','))
                df['total'] = pd.to_numeric(df['total'].replace('.',','))
                df['date'] = pd.to_datetime(df['date'])            
                df['day'] = pd.to_datetime(df['date'],format='%Y-%m-%d').dt.date
                
                array_dataframe.append(df)       
            else:
                print(entry.path)

dados = pd.concat(array_dataframe)

dados = dados.replace([np.inf, -np.inf], np.nan)
dados = dados.fillna(0)

dados.set_index('date', inplace=True)
dados.index = pd.to_datetime(dados.index)

dados.head()

Operacoes/USDT_BTC/2018-11.json
Operacoes/USDT_BTC/2018-12.json
Operacoes/USDT_ETH/2018-11.json
Operacoes/USDT_ETH/2018-12.json
Operacoes/USDT_LTC/2018-11.json
Operacoes/USDT_LTC/2018-12.json
Operacoes/USDT_ZEC/2018-11.json
Operacoes/USDT_ZEC/2018-12.json


,amount,globalTradeID,rate,total,tradeID,type,coin,day
date,,,,,,,,
2016-02-29 02:56:38,0.010240,18015876,4.464168,4.464168,84413,sell,USDT_BTC,2016-02-29
2016-02-29 02:56:36,0.014463,18015868,6.304939,6.304939,84412,sell,USDT_BTC,2016-02-29
2016-02-29 02:54:27,0.001508,18015626,0.657316,0.657316,84411,sell,USDT_BTC,2016-02-29
2016-02-29 02:52:27,0.180348,18015402,78.702099,78.702099,84410,buy,USDT_BTC,2016-02-29
2016-02-29 02:52:13,0.064586,18015379,28.184589,28.184589,84409,buy,USDT_BTC,2016-02-29


In [112]:
#Calcula a primeira data e a primeira janela de 24h

#data_inicial = dados.index.min()
data_inicial = pd.to_datetime('2015-08-01')
print(data_inicial)

#Calcula data final da primeira janela
data_final = data_inicial + timedelta(days=1)
print(data_final)

#Filtro para a janela
#mask_janela = (dados.index > data_inicial) & (dados.index <= data_final)

#Moeda base
moeda_base = 'USDT_BTC'

2015-08-01 00:00:00
2015-08-02 00:00:00


In [113]:
def obtem_janela(dt_inicial, dt_final):
    return dados.loc[(dados.index > dt_inicial) & (dados.index <= dt_final)]    

In [114]:
janela = obtem_janela(data_inicial,data_final)
print(janela)

                       amount  globalTradeID       rate      total  tradeID  \
date                                                                          
2015-08-01 17:48:05  0.010000        9240823   2.798450   2.798450    19626   
2015-08-01 17:43:27  0.000107        9240809   0.030301   0.030301    19625   
2015-08-01 17:43:27  0.012905        9240808   3.611372   3.611372    19624   
2015-08-01 17:38:59  0.224556        9240278  62.800543  62.800543    19623   
2015-08-01 17:37:52  0.172513        9240246  47.566018  47.566018    19622   
2015-08-01 17:37:49  0.360932        9240215  99.517826  99.517826    19621   
2015-08-01 17:37:49  0.018569        9240214   5.162249   5.162249    19620   
2015-08-01 17:37:49  0.000352        9240213   0.098417   0.098417    19619   
2015-08-01 17:37:49  0.010130        9240212   2.834320   2.834320    19618   
2015-08-01 17:37:49  0.010017        9240211   2.802650   2.802650    19617   
2015-08-01 10:03:19  0.046703        9232808  13.263

In [115]:
def avalia_moeda_base(dt):
    coin_result = dt[dt.coin == moeda_base]
    
    if not coin_result.empty  :
        dt_sell = coin_result[coin_result.type == 'sell']
        dt_buy  = coin_result[coin_result.type == 'buy']
    
        #Verifica se a moeda base está em queda (sell > buy)
        if dt_sell.amount.sum() > dt_buy.amount.sum() :
            #print('Moeda base em queda')
            return True
    
    #print('Base não está em queda')
    return False


def avalia_compra(dt):
    
    #print(dt)        
    ## Verifica se a moeda base está no momento de compra
    if avalia_moeda_base(dt):
        
        for c in dt.coin.unique():
            coin_result = dt[dt.coin == c]
            
            if not coin_result.empty  :
                dt_sell = coin_result[coin_result.type == 'sell']
                dt_buy  = coin_result[coin_result.type == 'buy']
    
                #Verifica se a moeda base está em queda (sell > buy)
                if dt_buy.amount.sum() > dt_sell.amount.sum():
                    return True
    
    return False
#def avalia_compra(dt):
#    if dt_sell.amount.sum() > dt_buy.amount.sum() :
#        return True
#    
#    return False

In [116]:
def simula_compra(dt):
    print('compra!')
    print(dt.day.max())

In [117]:
#RESET
data_inicial = pd.to_datetime('2015-08-01')
data_final = data_inicial + timedelta(days=1)

#Coleta a Janela
janela = obtem_janela(data_inicial,data_final)

while not janela.empty :

    if avalia_compra(janela):
        simula_compra(janela)
    
    data_inicial = data_final + timedelta(days=1)
    data_final = data_inicial + timedelta(days=1)
    
    print('De: '+ str(data_inicial) + 'Até: '+ str(data_final))
    janela = obtem_janela(data_inicial,data_final)   
    #input("Press Enter to continue...")

De: 2015-08-03 00:00:00Até: 2015-08-04 00:00:00
De: 2015-08-05 00:00:00Até: 2015-08-06 00:00:00
De: 2015-08-07 00:00:00Até: 2015-08-08 00:00:00
De: 2015-08-09 00:00:00Até: 2015-08-10 00:00:00
De: 2015-08-11 00:00:00Até: 2015-08-12 00:00:00
compra!
2015-08-11
De: 2015-08-13 00:00:00Até: 2015-08-14 00:00:00
compra!
2015-08-13
De: 2015-08-15 00:00:00Até: 2015-08-16 00:00:00
compra!
2015-08-15
De: 2015-08-17 00:00:00Até: 2015-08-18 00:00:00
De: 2015-08-19 00:00:00Até: 2015-08-20 00:00:00
compra!
2015-08-19
De: 2015-08-21 00:00:00Até: 2015-08-22 00:00:00
De: 2015-08-23 00:00:00Até: 2015-08-24 00:00:00
De: 2015-08-25 00:00:00Até: 2015-08-26 00:00:00
De: 2015-08-27 00:00:00Até: 2015-08-28 00:00:00
compra!
2015-08-27
De: 2015-08-29 00:00:00Até: 2015-08-30 00:00:00
De: 2015-08-31 00:00:00Até: 2015-09-01 00:00:00
De: 2015-09-02 00:00:00Até: 2015-09-03 00:00:00
De: 2015-09-04 00:00:00Até: 2015-09-05 00:00:00
De: 2015-09-06 00:00:00Até: 2015-09-07 00:00:00
De: 2015-09-08 00:00:00Até: 2015-09-09 00

De: 2016-06-10 00:00:00Até: 2016-06-11 00:00:00
De: 2016-06-12 00:00:00Até: 2016-06-13 00:00:00
De: 2016-06-14 00:00:00Até: 2016-06-15 00:00:00
De: 2016-06-16 00:00:00Até: 2016-06-17 00:00:00
De: 2016-06-18 00:00:00Até: 2016-06-19 00:00:00
De: 2016-06-20 00:00:00Até: 2016-06-21 00:00:00
De: 2016-06-22 00:00:00Até: 2016-06-23 00:00:00
De: 2016-06-24 00:00:00Até: 2016-06-25 00:00:00
De: 2016-06-26 00:00:00Até: 2016-06-27 00:00:00
compra!
2016-06-26
De: 2016-06-28 00:00:00Até: 2016-06-29 00:00:00
De: 2016-06-30 00:00:00Até: 2016-07-01 00:00:00
De: 2016-07-02 00:00:00Até: 2016-07-03 00:00:00
De: 2016-07-04 00:00:00Até: 2016-07-05 00:00:00
De: 2016-07-06 00:00:00Até: 2016-07-07 00:00:00
De: 2016-07-08 00:00:00Até: 2016-07-09 00:00:00
De: 2016-07-10 00:00:00Até: 2016-07-11 00:00:00
De: 2016-07-12 00:00:00Até: 2016-07-13 00:00:00
De: 2016-07-14 00:00:00Até: 2016-07-15 00:00:00
De: 2016-07-16 00:00:00Até: 2016-07-17 00:00:00
De: 2016-07-18 00:00:00Até: 2016-07-19 00:00:00
De: 2016-07-20 00:00: